In [1]:
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import json

In [16]:
# load model
model = load_model('model.h5')

# load scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# load label gender encoder
with open('label_encoder_gender.pkl','rb') as f:
     label_encoder_gender = pickle.load(f)

# load one hot encoder
with open('onehot_encoder_geo.pkl','rb') as f:
     label_encoder_geo = pickle.load(f)


In [17]:
label_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [32]:
# example input data
input_data = {
    'CreditScore': 500,
    'Geography': 'Germany',
    'Gender': 'Female',
    'Age': 19,
    'Tenure': 3,
    'Balance': 6000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 25000
}


In [33]:
# convert catogorical values into numerical values
# once hot encoding

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [34]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,Germany,Female,19,3,6000,2,1,1,25000


In [35]:
# convert catogorical values into numerical values
# encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
input_data['Gender'] = le.fit_transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,Germany,0,19,3,6000,2,1,1,25000


In [36]:
# concatination of one hot encoded
input_data = pd.concat([input_data.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,500,0,19,3,6000,2,1,1,25000,0.0,1.0,0.0


In [37]:
input_data.shape

(1, 12)

In [38]:
# scaling the data
input_data = scaler.transform(input_data)
input_data

array([[-1.57375827, -1.09499335, -1.89172757, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -1.31149897, -0.99850112,
         1.72572313, -0.57638802]])

In [39]:
# predict the output
prediction = model.predict(input_data)
prediction

1/1 [==============================] - 0s 17ms/step


array([[0.00039762]], dtype=float32)

In [40]:
# get the probability

prediction_proba = prediction[0][0]
prediction_proba

0.00039761563

In [41]:
if prediction_proba > 0.5:
    print('The customer is likely to churn..')
else:
    print('The customer is not likely to stay..')

The customer is not likely to stay..


In [42]:
label_encoder_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [43]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)